In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from scipy.stats import mode

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import train_test_split

In [ ]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier, XGBRFClassifier
from catboost import CatBoostClassifier

In [ ]:
from sklearn.ensemble import VotingClassifier, StackingClassifier, RandomForestClassifier

In [ ]:
train = pd.read_csv('../input/dsn-2021/Train.csv')
test= pd.read_csv('../input/dsn-2021/Test.csv')
submission = pd.read_csv("../input/dsn-2021/SampleSubmission.csv")

In [ ]:
train.columns

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
train.columns

In [ ]:
train.info()

In [ ]:
train.shape, test.shape

In [ ]:
(train.nunique() * 100)/1568

In [ ]:
target = train['Response']
dropping = ['ID']
drop_id = ['ID', 'Response']
train = train.drop(drop_id, axis=1)
test = test.drop(dropping, axis=1)

In [ ]:
train = train.drop_duplicates()
test = test.drop_duplicates()

In [ ]:
train.head()

In [ ]:
train.columns

In [ ]:
train.shape, test.shape

In [ ]:
target.shape

In [ ]:
train.describe()

In [ ]:
std_col = []
for col  in train.describe().columns:
    std = train.describe()[col]['std']
    if std > 50 and col != 'Response':
        std_col.append(col)

In [ ]:
len(std_col)

In [ ]:
concated = pd.concat([train,test])

In [ ]:
concated.Date_Customer = pd.to_datetime(concated.Date_Customer)

In [ ]:
year= concated.Date_Customer.dt.year
month= concated.Date_Customer.dt.month
day= concated.Date_Customer.dt.day

In [ ]:
concated= pd.get_dummies(concated)

In [ ]:
concated.head()

In [ ]:
concated = concated.drop(['Date_Customer'], axis = 1)

In [ ]:
ntrain = train.shape[0]
ntrain

In [ ]:
train_dummied= concated.iloc[:ntrain]
test_dummied= concated.iloc[ntrain:]

In [ ]:
train_dummied.shape, test_dummied.shape

In [ ]:
train.columns

In [ ]:
test.columns

In [ ]:
train.Education_Level

In [ ]:
train_dummied.columns

In [ ]:
train_dummied.Marital_Status_Widow.nunique()

new_train = train.drop(cat_cols, axis=1)
new_test = test.drop(cat_cols, axis=1)

In [ ]:
train_dummied.head()

new_train_dummied = train_dummied.fillna(0)
new_test_dummied = test_dummied.fillna(0)

new_train_dummied.isnull().sum()

In [ ]:
cols = train_dummied.columns.difference(['Response'])
X = train_dummied[cols]
y= target

In [ ]:
X.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state= 42, test_size= 0.3)

%%time
scaler = StandardScaler()

X_train[X.columns] = scaler.fit_transform(X_train[X.columns])
X_test[X.columns] = scaler.fit_transform(X_test[X.columns])

### LOGISTIC REGRESSION

lr = LogisticRegression()

lr.fit(X_train, y_train)

%%time
lr_y_pred = lr.predict(X_test)

print("f1_score")
f1_score(y_test, lr_y_pred)

test_pred_lr= lr.predict(new_test[X.columns])

test_pred_lr[:]

In [ ]:
states = [y for y in range(42, (42+20))]
for x in states:
    print(x)

### Catboost

In [ ]:
cbc_pred_=[]
states = [y for y in range(42, (42+20))]
for x in states:
    cbc= CatBoostClassifier(iterations= 2000, random_state= x)
    cbc.fit(X,y, eval_set= (X,y), use_best_model= True)
    cbc_pred = cbc.predict(test_dummied)
    cbc_pred_.append(cbc_pred)

In [ ]:
from scipy.stats import mode
mode_test_pred = mode(cbc_pred_)



mode_test_pred

In [ ]:
 cbc= CatBoostClassifier(iterations= 2000, random_state= 42)

In [ ]:
cbc.fit(X,y, eval_set= (X,y), use_best_model= True)

In [ ]:
sns.countplot(train.Response)

In [ ]:
cbc_pred = cbc.predict(test_dummied)

In [ ]:
X_train.shape, test_dummied.shape

In [ ]:
X_train.head()

In [ ]:
test_dummied.head()

### LGBM

In [ ]:
lgb_pred_=[]
states_ = [y for y in range(42, (42+20))]
for x in states_:
    lm2 = LGBMClassifier( random_state=x)
    lm2.fit(X,y)
    y_pred = lm2.predict(test_dummied[X.columns])
    lgb_pred_.append(y_pred)


lm2 = LGBMClassifier( random_state=42)
lm2.fit(X,y)
y_pred = lm2.predict(test_dummied[X.columns])

In [ ]:
lgb_mode_test_pred = mode(lgb_pred_)

In [ ]:
f1_score(y_test, y_pred)

In [ ]:
y= y.drop_duplicates()

In [ ]:
X.shape, y.shape

### XGBOOST

In [ ]:
xg_pred_=[]
state_ = [y for y in range(42, (42+20))]
for x in state_:
    xg = XGBClassifier(random_state=x )
    xg.fit(X,y)
    xg_pred = xg.predict(test_dummied[X.columns])
    xg_pred_.append(xg_pred)

In [ ]:
xgb_mode_test_pred = mode(xg_pred_)

### Voting Classifier

In [ ]:
cb_ = CatBoostClassifier(iterations= 2000, random_state= x)
xg_ = XGBClassifier(random_state=x )
lg_ = LGBMClassifier( random_state=x)

%%time
vot_pred=[]
state_ = [y for y in range(42, (42+20))]
for x in state_:
    vot_mod = VotingClassifier(estimators=[('cb_',cb_),('xg_',xg_),('lg_',lg_)])
    vot_mod.fit(X,y)
    vot_mod_pred = vot_mod.predict(test_dummied[X.columns])
    vot_pred.append(vot_mod_pred)

## Stacking

%%time
voti_pred=[]
state_ = [y for y in range(42, (42+20))]
for x in state_:
    voti_mod = StackingClassifier(estimators=[('cb_',cb_),('xg_',xg_),('lg_',lg_)], n_jobs = -1)
    voti_mod.fit(X,y)
    voti_mod_pred = voti_mod.predict(test_dummied[X.columns])
    voti_pred.append(voti_mod_pred)

In [ ]:
voti_pred_mod_ = mode(voti_pred)

### Submission

In [ ]:
submission.head()

submission['Response'] = mode_test_pred[0][0]

In [ ]:
vot_pred_mod_ = mode(vot_pred)

## Blending

In [ ]:
catb= pd.read_csv('../input/best-results/catboost_train_test_split_42_61_date_customer_dropped.csv')
light= pd.read_csv('../input/best-results/lgb_42_61.csv')
xboost = pd.read_csv('../input/best-results/xgb_random_state_42-61_date_included_splitted.csv')

In [ ]:
blend_ = mode([catb['Response'],light['Response'],xboost['Response']])

In [ ]:
submission['Response'] = blend_[0][0]

In [ ]:
submission.head()

In [ ]:
submission.to_csv('blend_cat_xgb_lgb.csv', index=False)